In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns 
import math
import statistics
import random

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, homogeneity_score

from collections import Counter

# Functions

In [2]:
def id_to_cluster(linkage_data, agg_features_df):
    """
    linkage_data: array of cluster numbers
    agg_features_df: df of aggregated features to merge, index must match order of linkage_data
    Returns a df with aggregated cp data and cluster number for each id
    """
    # Get cluster number with the aggregated feature data
    clusters_hierarchal_df = pd.DataFrame(data=linkage_data, index=agg_features_df.index)
    clusters_hierarchal_df.rename(columns={0:'cluster_num'}, inplace=True)
    clusters_hierarchal_df = clusters_hierarchal_df.merge(agg_features_df, how='left', left_index=True, right_index=True)

    # Get the cell profiler features by cluster
    cp_features_by_cluster = clusters_hierarchal_df.groupby(by='cluster_num').mean()
    
    return clusters_hierarchal_df

def hierarchical_cluster(df, threshold, show=False):
    """
    Returns df of cluster features
    """
    threshold=threshold
    Z1 = linkage(df, 'ward')
    flat_linkage1 = fcluster(Z1, t=threshold, criterion='distance')
    cluster_features1 = id_to_cluster(flat_linkage1, df)
    
    if show:
        plt.figure(figsize=(12, df.shape[0]/4))
        plt.axvline(x=threshold)
        label = [str(i) + ' ' + j for i, j in zip(flat_linkage1.tolist(), df.index.tolist())]
        plt.title('Clusters based on cell profiler features')
        dend1 = dendrogram(Z1, color_threshold=threshold, orientation='left', leaf_font_size=10, labels=label)
    return cluster_features1

def km_cluster(df, num_clusters, random_state=2):
    """
    returns df of cluster features
    """
    km = KMeans(n_clusters=num_clusters, random_state=random_state).fit(df)
    return id_to_cluster(km.labels_, df)

In [3]:
def ttest_2_df(df1, df2, alpha=0.05, reject_only=True):
    """
    Inputs are 2 dataframes with the same column features. Returns FDR adjusted p-values for each feature.
    """
#     assert df1.columns == df2.columns
    z_score, pvals = ttest_ind(df1.values, df2.values, axis=0)
    p_df = pd.DataFrame(pvals, index = df1.columns)
    p_df = p_df.rename(columns={0: 'pval'}) # unadjusted p values
    
    # Remove first row, which is just a ttest on the column number
    p_df =  p_df.iloc[1:]
    
    # Get corrected p-values
    q_df = p_df
    q_df['adj_p_value'] = multipletests(p_df['pval'].values, alpha=alpha, method='fdr_bh')[1]
    q_df['reject_null'] = multipletests(p_df['pval'].values, alpha=alpha, method='fdr_bh')[0]
    
    if reject_only:
        q_df = q_df[q_df['reject_null'] == True]
    
    return q_df

# Load data

In [20]:
# Progenitors data
from pathlib import Path
path = os.getcwd()
base_dir = str(Path(path).parent)

switch_isogenic_labels = True
human_only=True

#FS data
progenitors = pd.read_csv(base_dir + '/feature_sets/Progenitors/0714_stdev_corr_fs.csv', index_col=0)

exclude = ['5', '6', '33', '12', '16']
try:
    idx = [i for i in progenitors.index.tolist() if i.split('_')[2] not in exclude] # Exclude patient number
    progenitors = progenitors[progenitors.index.isin(idx)]
except: pass

if human_only:
    progenitors = progenitors[progenitors.index.str.contains('human')]

print ('shape: {}'.format(progenitors.shape))

labels = progenitors.index.tolist()
if switch_isogenic_labels:
    for i in range(len(labels)):
        if 'isogenic_deletion' in labels[i]:
            labels[i] = labels[i].replace('isogenic_deletion', 'temp')
    for i in range(len(labels)):
        if 'isogenic_control' in labels[i]:
            labels[i] = labels[i].replace('isogenic_control', 'isogenic_deletion')            
        if 'temp' in labels[i]:
            labels[i] = labels[i].replace('temp', 'isogenic_control')                    
progenitors.index = labels
progenitors_orig = progenitors.copy()

shape: (312, 508)


In [5]:
# STEM data
# Or use my FS data
stem = pd.read_csv(base_dir + '/feature_sets/STEM01/0621_stdev_corr_fs.csv', index_col=0)


try:
    idx = [i for i in stem.index.tolist() if i.split('_')[-1] not in exclude]
    stem = stem[stem.index.isin(idx)]
except: pass

if human_only:
    stem = stem[stem.index.str.contains('human')]
    
print ('shape: {}'.format(stem.shape))

labels = stem.index.tolist()
if switch_isogenic_labels:
    for i in range(len(labels)):
        if 'isogenic_deletion' in labels[i]:
            labels[i] = labels[i].replace('isogenic_deletion', 'temp')
    for i in range(len(labels)):
        if 'isogenic_control' in labels[i]:
            labels[i] = labels[i].replace('isogenic_control', 'isogenic_deletion')            
        if 'temp' in labels[i]:
            labels[i] = labels[i].replace('temp', 'isogenic_control')                    
stem.index = labels
stem_orig = stem.copy()

shape: (352, 559)


# Logistic regression and decision trees

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans

In [7]:
stem['label'] = stem.index.str.split('_').map(lambda x: x[1])
stem['sample_num'] = stem.index.str.split('_').map(lambda x: x[2])

all_nums = stem['sample_num']
stem_xtrain, stem_xtest, stem_ytrain, stem_ytest = train_test_split(stem.drop(['label'], axis=1), stem['label'], 
                                                                    test_size=0.3, random_state=4)
print(len(stem_xtrain) + len(stem_xtest))

logr1 = LogisticRegression(random_state=2, penalty='l2', C=1).fit(stem_xtrain, stem_ytrain)
print(logr1.score(stem_xtrain, stem_ytrain), logr1.score(stem_xtest, stem_ytest))
# print(logr1.score(stem_xtrain, stem_ytrain.sample(frac=1)), logr1.score(stem_xtest, stem_ytest.sample(frac=1)))

352
1.0 1.0


In [21]:
progenitors['label'] = progenitors.index.str.split('_').map(lambda x: x[1])
progenitors_xtrain, progenitors_xtest, progenitors_ytrain, progenitors_ytest = train_test_split(progenitors.drop(['label'], axis=1), progenitors['label'], 
                                                                    test_size=0.3, random_state=4)
print(len(progenitors_xtrain) + len(progenitors_xtest))

logr2 = LogisticRegression(random_state=2, penalty='l2', C=1, max_iter=130).fit(progenitors_xtrain, progenitors_ytrain)
print(logr2.score(progenitors_xtrain, progenitors_ytrain), logr2.score(progenitors_xtest, progenitors_ytest))
# print(logr2.score(progenitors_xtrain, progenitors_ytrain.sample(frac=1)), logr2.score(progenitors_xtest, progenitors_ytest.sample(frac=1)))

312
1.0 0.8936170212765957


In [9]:
# hclus_stem = hierarchical_cluster(stem.drop(['label'], axis=1), 200, show=False)
# hclus_prog = hierarchical_cluster(progenitors.drop('label', axis=1), 170, show=True)

# q = hclus_stem[hclus_stem['cluster_num']==2]
# q[q.index.str.contains('del')]

In [10]:
tree1 = DecisionTreeClassifier(random_state=0).fit(stem_xtrain, stem_ytrain)
print(tree1.score(stem_xtrain, stem_ytrain), tree1.score(stem_xtest, stem_ytest))
print(tree1.score(stem_xtrain, stem_ytrain.sample(frac=1)), tree1.score(stem_xtest, stem_ytest.sample(frac=1)))

1.0 1.0
0.4878048780487805 0.5094339622641509


In [11]:
tree2 = DecisionTreeClassifier(random_state=0).fit(progenitors_xtrain, progenitors_ytrain)
print(tree2.score(progenitors_xtrain, progenitors_ytrain), tree2.score(progenitors_xtest, progenitors_ytest))
print(tree2.score(progenitors_xtrain, progenitors_ytrain.sample(frac=1)), tree2.score(progenitors_xtest, progenitors_ytest.sample(frac=1)))

1.0 0.8380952380952381
0.48148148148148145 0.47619047619047616


In [12]:
stem = stem.drop(['label'], axis=1)
progenitors = progenitors.drop(['label'], axis=1)

# Feature Importance

In [13]:
stem_feature_weights = pd.DataFrame(logr1.coef_[0], columns=['weight'], index=stem.columns)
stem_feature_weights.reindex(stem_feature_weights['weight'].abs().sort_values(ascending=False).index).head(15)

,weight
sample_num,0.684903
Nuclei_Texture_InfoMeas1_DNA_10_01,0.089962
Cytoplasm_Intensity_MADIntensity_ER,0.069861
Cells_Intensity_LowerQuartileIntensity_Mito,0.069280
Cytoplasm_RadialDistribution_FracAtD_Brightfield_1of4,0.068099
Nuclei_Correlation_Correlation_DNA_Brightfield,0.066358
Nuclei_Correlation_Correlation_DNA_AGP,0.065501
Cells_Intensity_MedianIntensity_Mito,0.064868
Cytoplasm_AreaShape_EulerNumber,0.060670
Cytoplasm_RadialDistribution_FracAtD_Brightfield_4of4,-0.060202


In [14]:
stem_tree_feature_weights = pd.DataFrame(tree1.feature_importances_, columns=['weight'], index=stem.columns)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(stem_tree_feature_weights.sort_values(by='weight', ascending=False).head(13))


,weight
sample_num,1.0
Cytoplasm_RadialDistribution_FracAtD_DNA_4of4,0.0
Cells_RadialDistribution_MeanFrac_DNA_3of4,0.0
Cells_RadialDistribution_RadialCV_DNA_1of4,0.0
Cells_RadialDistribution_RadialCV_DNA_2of4,0.0
Cells_RadialDistribution_RadialCV_DNA_3of4,0.0
Cells_RadialDistribution_RadialCV_DNA_4of4,0.0
Cytoplasm_RadialDistribution_FracAtD_DNA_1of4,0.0
Cytoplasm_RadialDistribution_MeanFrac_DNA_4of4,0.0
Cells_RadialDistribution_FracAtD_DNA_3of4,0.0


In [15]:
ttest_stem = ttest_2_df(stem[stem.index.str.contains('control')], stem[stem.index.str.contains('deletion')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ttest_stem.sort_values(by='adj_p_value').head(15))

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
progenitors_feature_weights = pd.DataFrame(logr2.coef_[0], columns=['weight'], index=progenitors.columns)
progenitors_feature_weights.reindex(progenitors_feature_weights['weight'].abs().sort_values(ascending=False).index)

In [ ]:
progenitors_tree_feature_weights = pd.DataFrame(tree2.feature_importances_, columns=['weight'], index=progenitors.columns)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(progenitors_tree_feature_weights.sort_values(by='weight', ascending=False).head(13))

In [ ]:
ttest_progenitors = ttest_2_df(progenitors[progenitors.index.str.contains('control')], progenitors[progenitors.index.str.contains('deletion')])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ttest_progenitors.sort_values(by='adj_p_value'))

# Feature distribution figures

In [ ]:
# Stem logistic

features = ['Cytoplasm_RadialDistribution_FracAtD_Brightfield_1of4',
 'Nuclei_Correlation_Correlation_DNA_Brightfield',
 'Cytoplasm_RadialDistribution_FracAtD_Brightfield_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_1of4',
 'Cytoplasm_Texture_InfoMeas1_AGP_5_00',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Nuclei_Granularity_13_AGP',
 'Nuclei_AreaShape_Zernike_8_0',
 'Cytoplasm_AreaShape_Zernike_3_1',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_4of4',
 'Cells_Granularity_1_ER',
 'Cytoplasm_Texture_InfoMeas1_AGP_10_00',
 'Nuclei_Texture_InfoMeas1_DNA_10_01',
 'Cells_Correlation_Overlap_ER_AGP']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    ctrl_vals = human_ctrl_df[[feature]]
    del_vals = human_del_df[[feature]]
    
    sns.histplot(np.ravel(ctrl_vals), kde=True, ax=axs[row, col], color='blue')
    sns.histplot(np.ravel(del_vals), kde=True, ax=axs[row, col], color='red')
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
        
plt.suptitle('Most important features, stem, logistic regression', size=30)
plt.legend(handles=[mpatches.Patch(color='red', label='human deletion'), 
           mpatches.Patch(color='blue', label='human control')])

In [ ]:
# Stem ttest

features = ['Cells_Correlation_Overlap_Mito_AGP',
 'Cells_RadialDistribution_RadialCV_Mito_1of4',
 'Cells_Granularity_2_Mito',
 'Cells_Correlation_Overlap_DNA_Mito',
 'Cells_Correlation_Overlap_Mito_Brightfield',
 'Cells_Granularity_10_RNA',
 'Cells_Correlation_Overlap_ER_Brightfield',
 'Cells_RadialDistribution_FracAtD_AGP_4of4',
 'Cells_Granularity_10_ER',
 'Nuclei_Intensity_MassDisplacement_Mito',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Cells_RadialDistribution_FracAtD_AGP_1of4',
 'Cells_RadialDistribution_MeanFrac_Mito_3of4',
 'Cells_Granularity_2_AGP',
 'Cells_Correlation_Overlap_AGP_Brightfield']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    ctrl_vals = human_ctrl_df[[feature]]
    del_vals = human_del_df[[feature]]
    
    sns.histplot(np.ravel(ctrl_vals), kde=True, ax=axs[row, col], color='blue')
    sns.histplot(np.ravel(del_vals), kde=True, ax=axs[row, col], color='red')
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
        
plt.suptitle('Most important features, stem, t-test', size=30)
plt.legend(handles=[mpatches.Patch(color='red', label='human deletion'), 
           mpatches.Patch(color='blue', label='human control')])

In [ ]:
# Stem tree

features = ['Cells_Correlation_Overlap_Mito_AGP',
 'Cells_RadialDistribution_FracAtD_ER_4of4',
 'Cells_Correlation_Overlap_ER_Brightfield',
 'Nuclei_Correlation_Correlation_ER_AGP',
 'Cells_Intensity_LowerQuartileIntensity_ER',
 'Nuclei_Texture_Correlation_ER_10_00',
 'Nuclei_Texture_AngularSecondMoment_Brightfield_20_01',
 'Cells_Texture_InverseDifferenceMoment_ER_10_00',
 'Cells_Texture_SumAverage_ER_10_00',
 'Cells_Granularity_4_AGP']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(2,5, figsize=(30,12))
row = 0
col = 0
for feature in features:
    ctrl_vals = human_ctrl_df[[feature]]
    del_vals = human_del_df[[feature]]
    
    sns.histplot(np.ravel(ctrl_vals), kde=True, ax=axs[row, col], color='blue')
    sns.histplot(np.ravel(del_vals), kde=True, ax=axs[row, col], color='red')
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
        
plt.suptitle('Most important features, stem, tree', size=30)
plt.legend(handles=[mpatches.Patch(color='red', label='human deletion'), 
           mpatches.Patch(color='blue', label='human control')])

In [ ]:
# Progenitors logistic

features = ['Cells_Correlation_Overlap_ER_AGP',
 'Nuclei_Correlation_Correlation_Mito_AGP',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_1of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_1of4',
 'Nuclei_Correlation_Correlation_DNA_RNA',
 'Cells_Correlation_Overlap_Mito_ER',
 'Nuclei_Granularity_7_DNA',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_2of4',
 'Cells_Correlation_Overlap_RNA_AGP',
 'Cells_Correlation_Correlation_Mito_AGP',
 'Nuclei_Granularity_10_DNA',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_1of4',
 'Nuclei_Intensity_MassDisplacement_DNA',
 'Cells_Correlation_Correlation_DNA_RNA']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    ctrl_vals = human_ctrl_df[[feature]]
    del_vals = human_del_df[[feature]]
    
    sns.histplot(np.ravel(ctrl_vals), kde=True, ax=axs[row, col], color='blue')
    sns.histplot(np.ravel(del_vals), kde=True, ax=axs[row, col], color='red')
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
        
plt.suptitle('Most important features, progenitors, logistic regression', size=30)
plt.legend(handles=[mpatches.Patch(color='red', label='human deletion'), 
           mpatches.Patch(color='blue', label='human control')])

In [ ]:
# Progenitors ttest

features = ['Cells_RadialDistribution_RadialCV_Brightfield_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_2of4',
 'Cells_Texture_InfoMeas1_AGP_20_01',
 'Nuclei_Correlation_K_RNA_Brightfield',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_1of4',
 'Cells_Texture_Correlation_ER_20_01',
 'Cells_RadialDistribution_MeanFrac_Brightfield_3of4',
 'Cells_Correlation_Overlap_ER_AGP',
 'Cells_RadialDistribution_MeanFrac_Brightfield_4of4',
 'Cells_Texture_Correlation_DNA_10_00',
 'Cytoplasm_Texture_InfoMeas1_RNA_10_00',
 'Nuclei_Correlation_K_Brightfield_RNA',
 'Cells_Texture_Correlation_RNA_20_01',
 'Cytoplasm_Texture_Correlation_Brightfield_20_03',
 'Cells_Correlation_K_RNA_Brightfield']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    ctrl_vals = human_ctrl_df[[feature]]
    del_vals = human_del_df[[feature]]
    
    sns.histplot(np.ravel(ctrl_vals), kde=True, ax=axs[row, col], color='blue')
    sns.histplot(np.ravel(del_vals), kde=True, ax=axs[row, col], color='red')
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
        
plt.suptitle('Most important features, progenitors, t-test', size=30)
plt.legend(handles=[mpatches.Patch(color='red', label='human deletion'), 
           mpatches.Patch(color='blue', label='human control')])

In [ ]:
# Prog tree

features = ['Cells_Texture_Correlation_RNA_20_01',
 'Cytoplasm_RadialDistribution_MeanFrac_AGP_4of4',
 'Nuclei_Correlation_Correlation_ER_AGP',
 'Cytoplasm_RadialDistribution_FracAtD_ER_3of4',
 'Nuclei_Correlation_K_AGP_DNA',
 'Cells_Correlation_Overlap_DNA_ER',
 'Cells_RadialDistribution_RadialCV_Mito_1of4',
 'Cells_Texture_InfoMeas2_RNA_10_00',
 'Cells_AreaShape_Eccentricity',
 'Cells_Granularity_7_Brightfield']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(2,5, figsize=(30,12))
row = 0
col = 0
for feature in features:
    ctrl_vals = human_ctrl_df[[feature]]
    del_vals = human_del_df[[feature]]
    
    sns.histplot(np.ravel(ctrl_vals), kde=True, ax=axs[row, col], color='blue')
    sns.histplot(np.ravel(del_vals), kde=True, ax=axs[row, col], color='red')
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
        
plt.suptitle('Most important features, stem, tree', size=30)
plt.legend(handles=[mpatches.Patch(color='red', label='human deletion'), 
           mpatches.Patch(color='blue', label='human control')])

# Feature vs cell count/area

In [ ]:
# Merge stem counts with stem
stem_cts = pd.read_csv(base_dir + '/1.run-workflows/profiles/NCP_STEM_1/BR_NCP_STEM_1/BR_NCP_STEM_1_count.csv.gz', index_col=0)
stem = stem_orig
stem['well'] = stem.index.str.split('_').map(lambda x: x[-1])
stem['idx'] = stem.index
stem = stem.merge(stem_cts, how='inner', left_on='well', right_on='Metadata_Well').drop(['well', 'Metadata_Well'], axis=1)
stem.set_index('idx', inplace=True)

In [ ]:
# Merge progenitors with cell_number_object_number
progenitors = progenitors_orig.copy()
prog_cts = pd.read_csv(base_dir + '/1.run-workflows/profiles/NCP_PROGENITORS_1/BR_NCP_PROGENITORS_1.csv.gz', index_col=0)
prog_cts = prog_cts[['Metadata_Well', 'Cells_Number_Object_Number']]

progenitors['well'] = progenitors.index.str.split('_').map(lambda x: x[-1])
progenitors['well'] = progenitors.index.str.split('_').map(lambda x: x[-1])
progenitors['idx'] = progenitors.index
progenitors = progenitors.merge(prog_cts, how='inner', left_on='well', right_on='Metadata_Well').drop(['well', 'Metadata_Well'], axis=1)
progenitors.set_index('idx', inplace=True)

In [ ]:
# Stem logistic

features = ['Cytoplasm_RadialDistribution_FracAtD_Brightfield_1of4',
 'Nuclei_Correlation_Correlation_DNA_Brightfield',
 'Cytoplasm_RadialDistribution_FracAtD_Brightfield_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_1of4',
 'Cytoplasm_Texture_InfoMeas1_AGP_5_00',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Nuclei_Granularity_13_AGP',
 'Nuclei_AreaShape_Zernike_8_0',
 'Cytoplasm_AreaShape_Zernike_3_1',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_4of4',
 'Cells_Granularity_1_ER',
 'Cytoplasm_Texture_InfoMeas1_AGP_10_00',
 'Nuclei_Texture_InfoMeas1_DNA_10_01',
 'Cells_Correlation_Overlap_ER_AGP']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(stem['Count_Cells'], stem[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Cell Count', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, stem, logistic regression', size=30)


In [ ]:
# Stem t-test

features = ['Cells_Correlation_Overlap_Mito_AGP',
 'Cells_RadialDistribution_RadialCV_Mito_1of4',
 'Cells_Granularity_2_Mito',
 'Cells_Correlation_Overlap_DNA_Mito',
 'Cells_Correlation_Overlap_Mito_Brightfield',
 'Cells_Granularity_10_RNA',
 'Cells_Correlation_Overlap_ER_Brightfield',
 'Cells_RadialDistribution_FracAtD_AGP_4of4',
 'Cells_Granularity_10_ER',
 'Nuclei_Intensity_MassDisplacement_Mito',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Cells_RadialDistribution_FracAtD_AGP_1of4',
 'Cells_RadialDistribution_MeanFrac_Mito_3of4',
 'Cells_Granularity_2_AGP',
 'Cells_Correlation_Overlap_AGP_Brightfield']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(stem['Count_Cells'], stem[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Cell Count', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, stem, t-test', size=30)

In [ ]:
# Stem logistic

features = ['Cytoplasm_RadialDistribution_FracAtD_Brightfield_1of4',
 'Nuclei_Correlation_Correlation_DNA_Brightfield',
 'Cytoplasm_RadialDistribution_FracAtD_Brightfield_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_1of4',
 'Cytoplasm_Texture_InfoMeas1_AGP_5_00',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Nuclei_Granularity_13_AGP',
 'Nuclei_AreaShape_Zernike_8_0',
 'Cytoplasm_AreaShape_Zernike_3_1',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_4of4',
 'Cells_Granularity_1_ER',
 'Cytoplasm_Texture_InfoMeas1_AGP_10_00',
 'Nuclei_Texture_InfoMeas1_DNA_10_01',
 'Cells_Correlation_Overlap_ER_AGP']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(stem['Cells_AreaShape_Area'], stem[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Area', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, stem, logistic regression', size=30)

In [ ]:
# Stem t-test

features = ['Cells_Correlation_Overlap_Mito_AGP',
 'Cells_RadialDistribution_RadialCV_Mito_1of4',
 'Cells_Granularity_2_Mito',
 'Cells_Correlation_Overlap_DNA_Mito',
 'Cells_Correlation_Overlap_Mito_Brightfield',
 'Cells_Granularity_10_RNA',
 'Cells_Correlation_Overlap_ER_Brightfield',
 'Cells_RadialDistribution_FracAtD_AGP_4of4',
 'Cells_Granularity_10_ER',
 'Nuclei_Intensity_MassDisplacement_Mito',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Cells_RadialDistribution_FracAtD_AGP_1of4',
 'Cells_RadialDistribution_MeanFrac_Mito_3of4',
 'Cells_Granularity_2_AGP',
 'Cells_Correlation_Overlap_AGP_Brightfield']

human_ctrl_df = stem[stem.index.str.contains('human_control')]
human_del_df = stem[stem.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(stem['Cells_AreaShape_Area'], stem[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Area', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, stem, t-test', size=30)

In [ ]:
# Prog logistic count

features = ['Cells_Correlation_Overlap_ER_AGP',
 'Nuclei_Correlation_Correlation_Mito_AGP',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_1of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_1of4',
 'Nuclei_Correlation_Correlation_DNA_RNA',
 'Cells_Correlation_Overlap_Mito_ER',
 'Nuclei_Granularity_7_DNA',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_2of4',
 'Cells_Correlation_Overlap_RNA_AGP',
 'Cells_Correlation_Correlation_Mito_AGP',
 'Nuclei_Granularity_10_DNA',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_1of4',
 'Nuclei_Intensity_MassDisplacement_DNA',
 'Cells_Correlation_Correlation_DNA_RNA']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(progenitors['Cells_Number_Object_Number'], progenitors[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Cells_Number_Object_Number', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, progenitors, logistic regression', size=30)

In [ ]:
# Prog t-test count

features = ['Cells_RadialDistribution_RadialCV_Brightfield_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_2of4',
 'Cells_Texture_InfoMeas1_AGP_20_01',
 'Nuclei_Correlation_K_RNA_Brightfield',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_1of4',
 'Cells_Texture_Correlation_ER_20_01',
 'Cells_RadialDistribution_MeanFrac_Brightfield_3of4',
 'Cells_Correlation_Overlap_ER_AGP',
 'Cells_RadialDistribution_MeanFrac_Brightfield_4of4',
 'Cells_Texture_Correlation_DNA_10_00',
 'Cytoplasm_Texture_InfoMeas1_RNA_10_00',
 'Nuclei_Correlation_K_Brightfield_RNA',
 'Cells_Texture_Correlation_RNA_20_01',
 'Cytoplasm_Texture_Correlation_Brightfield_20_03',
 'Cells_Correlation_K_RNA_Brightfield']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(progenitors['Cells_Number_Object_Number'], progenitors[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Cells_Number_Object_Number', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, progenitors, t-test', size=30)

In [ ]:
# Prog logistic Area

features = ['Cells_Correlation_Overlap_ER_AGP',
 'Nuclei_Correlation_Correlation_Mito_AGP',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_1of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_1of4',
 'Nuclei_Correlation_Correlation_DNA_RNA',
 'Cells_Correlation_Overlap_Mito_ER',
 'Nuclei_Granularity_7_DNA',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_2of4',
 'Cells_Correlation_Overlap_RNA_AGP',
 'Cells_Correlation_Correlation_Mito_AGP',
 'Nuclei_Granularity_10_DNA',
 'Cytoplasm_RadialDistribution_MeanFrac_Brightfield_1of4',
 'Nuclei_Intensity_MassDisplacement_DNA',
 'Cells_Correlation_Correlation_DNA_RNA']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(progenitors['Cells_AreaShape_Area'], progenitors[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Cells_AreaShape_Area', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, progenitors, logistic regression', size=30)

In [ ]:
# Prog t-test area

features = ['Cells_RadialDistribution_RadialCV_Brightfield_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_2of4',
 'Cells_Texture_InfoMeas1_AGP_20_01',
 'Nuclei_Correlation_K_RNA_Brightfield',
 'Cytoplasm_RadialDistribution_RadialCV_Brightfield_1of4',
 'Cells_Texture_Correlation_ER_20_01',
 'Cells_RadialDistribution_MeanFrac_Brightfield_3of4',
 'Cells_Correlation_Overlap_ER_AGP',
 'Cells_RadialDistribution_MeanFrac_Brightfield_4of4',
 'Cells_Texture_Correlation_DNA_10_00',
 'Cytoplasm_Texture_InfoMeas1_RNA_10_00',
 'Nuclei_Correlation_K_Brightfield_RNA',
 'Cells_Texture_Correlation_RNA_20_01',
 'Cytoplasm_Texture_Correlation_Brightfield_20_03',
 'Cells_Correlation_K_RNA_Brightfield']

human_ctrl_df = progenitors[progenitors.index.str.contains('human_control')]
human_del_df = progenitors[progenitors.index.str.contains('human_deletion')]

fig, axs = plt.subplots(3,5, figsize=(30,18))
row = 0
col = 0
for feature in features:
    axs[row, col].scatter(progenitors['Cells_AreaShape_Area'], progenitors[feature])
    axs[row, col].set_title(feature)
    if col != 4:
        col += 1
    else:
        row += 1
        col = 0
fig.add_subplot(111, frameon=False)
plt.xlabel('Cells_AreaShape_Area', size=20)
plt.ylabel('Feature', size=20)
plt.suptitle('Most important features, progenitors, t-test', size=30)

In [ ]:
progenitors[['Cells_AreaShape_Area', 'Cells_Number_Object_Number']].sort_values(by='Cells_AreaShape_Area', ascending=False).head(20)

In [ ]:
progenitors[['Cells_AreaShape_Area', 'Cells_Number_Object_Number']].sort_values(by='Cells_AreaShape_Area', ascending=False).tail(20)

In [ ]:
progenitors[['Cells_AreaShape_Area', 'Cells_Number_Object_Number']].sort_values(by='Cells_Number_Object_Number', ascending=True).head(30)

In [ ]:
sns.displot(progenitors['Cells_Number_Object_Number'])